In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import warnings

from datetime import datetime

from common import *
from mcs_shared import (
    ACCUMULATION_FLIGHTS, SnotelPointData,
    load_als_depth, load_factors_tif,
    get_station_pixel_factors, get_station_pixel_depths
)

%load_ext autoreload
%autoreload 2

use_hvplot()

RESOLUTION = 10 # meters

## ALS depth - Min/Max normalization Center around 1
$factor = 2 * \frac{depth_i - X_{min}}{X_{max} - X_{min}}$

a: lower bound  
b: upper bound  

General form $ f = (b - a) \frac{x - min}{max - min} - a$

In [ ]:
als_factor_nd = [
    load_als_depth(flight, RESOLUTION, base_run=True) for flight in ACCUMULATION_FLIGHTS
]

#### Clip outliers outside 1st, 99th percentile

In [ ]:
for index, factor in enumerate(als_factor_nd):
    als_factor_nd[index] = np.clip(
        factor, 
        a_min=np.nanpercentile(factor, 1),
        a_max=np.nanpercentile(factor, 99)
    )

#### Normalize

In [ ]:
for index, factor in enumerate(als_factor_nd):
    als_factor_nd[index] = (
        2 * ((factor - np.nanmin(factor)) / (np.nanmax(factor) - np.nanmin(factor)))
    )

## Factors relative to MCS across seasons

In [ ]:
als_factor_norm = np.nanmedian(als_factor_nd, axis=0)

In [ ]:
hv.Image(als_factor_norm).opts(
    width=640, height=640, aspect='equal', colorbar=True, 
    cmap='HighContrast', clim=(0, 2), 
    tools=['hover']
)

In [ ]:
hist_data = als_factor_norm.flatten()
hist_data = hist_data[~np.isnan(hist_data)]

hv.Distribution(hist_data).opts(filled=False, width=800, height=600, tools=['hover'])

In [ ]:
def areal_plots(flight):
    maps = []
    x_coords = np.arange(0, als_factor_nd.shape[2] * RESOLUTION, RESOLUTION)
    y_coords = np.arange(0, als_factor_nd.shape[1] * RESOLUTION, RESOLUTION)

    hv_opts=dict(
        tools=['hover'],
        height=600, width=600, aspect='equal',
        colorbar=True,
        invert_yaxis=True
    )
    
    return hv.Layout(
            hv.Image(
                (x_coords, y_coords, load_als_depth(flight, RESOLUTION)),
            ).opts(
                title=pd.to_datetime(flight).strftime('%Y-%m-%d'),
                hover_tooltips=[('Depth', '@image')], cmap='PuBu', 
                clim=(0, 3.5), **hv_opts
            ) +
            hv.Image(
                (x_coords, y_coords, load_isnobal_depth(flight, RESOLUTION))
            ).opts(
                hover_tooltips=[('Depth', '@image')], cmap='PuBu', 
                clim=(0, 3.5), **hv_opts
            ) +
            hv.Image(
                (x_coords, y_coords, als_factor_norm - load_factors_tif(flight, RESOLUTION)), 
            ).opts(
                hover_tooltips=[
                    ("Factor", "@image{0.2f}"),
                    ("X", "$x{0f}"),
                    ("Y", "$y{0f}")
                ], 
                cmap='RdBu', clim=(-.5, .5), **hv_opts
            )
        ).cols(3)

for flight in ACCUMULATION_FLIGHTS:
    display(areal_plots(flight))

## Save via GDAL

In [ ]:
als_factor_norm[np.isnan(als_factor_norm)] = 0

In [ ]:
import random
from osgeo import gdal
gdal.UseExceptions()

In [ ]:
driver = gdal.GetDriverByName('GTiff')
src_ds = gdal.Open(f"/bsushare/hpmarshall-shared/jmeyer/MCS-ALS-snowdepth/{RESOLUTION}m_base/MCS_REFDEM_32611_{RESOLUTION}m.tif", gdal.GA_ReadOnly)

orig_file = driver.CreateCopy('/vsimem/orig_%i.tif' % random.getrandbits(32), src_ds)
out_band = orig_file.GetRasterBand(1)
out_band.WriteArray(als_factor_norm)
out_band.SetMetadataItem('NODATA', "0")
out_band.FlushCache()

# Fille gap in 10m source
filled_file = '/vsimem/filled_%i.tif' % random.getrandbits(32)
filled_options = gdal.TranslateOptions(
    xRes=1, yRes=1,
    resampleAlg=gdal.GRA_Average,
)
filled_ds = gdal.Translate(filled_file, orig_file, options=filled_options)

# Smooth to 100m for length scale
smooth_file = '/vsimem/smooth_%i.tif' % random.getrandbits(32)
smooth_options = gdal.TranslateOptions(
    xRes=100, yRes=100,
    resampleAlg=gdal.GRA_CubicSpline,
)
smooth_ds =  gdal.Translate(smooth_file, filled_ds , options=smooth_options)

# Store final map at model native resolution
pattern_file = f"/bsushare/hpmarshall-shared/jmeyer/MCS-ALS-snowdepth/precip_factors/MCS_pattern_{RESOLUTION}m.tif"
pattern_options = gdal.TranslateOptions(
    xRes=10, yRes=10,
    resampleAlg=gdal.GRA_NearestNeighbour,
)
pattern_ds =  gdal.Translate(pattern_file, smooth_ds, options=pattern_options)

src_ds = None
filled_ds = None
smooth_ds = None
pattern_ds = None

In Terminal:  
* Get back to model resolution
`gdalwarp -overwrite -co BAND_NAMES="pattern" -tr 10 10 -te 594356.438 4855619.000 616456.438 4877419.000 MCS_pattern_10m.tif MCS_HRRR_ALS_pattern.nc`
* Change values less than 0.01 to 0, these are most likely from interpolation around the edges
`cdo setrtoc,-inf,0.01,0`

In [ ]:
hrrr_nc = xr.open_dataset("/bsushare/hpmarshall-shared/jmeyer/MCS-ALS-snowdepth/precip_factors/MCS_HRRR_ALS_pattern.nc")

In [ ]:
hrrr_nc

In [ ]:
hrrr_nc.pattern.hvplot(height=600, width=600, aspect='equal', cmap='PuOr', clim=(0, 2))

In [ ]:
hist_data_nc = hrrr_nc.pattern.values.flatten()
hist_data_nc = hist_data_nc[~np.isnan(hist_data_nc)]

hv.Distribution(hist_data_nc).opts(filled=False, width=800, height=600, tools=['hover'])

In [ ]:
hrrr_nc.close()